In [ ]:
# path to our tables in the silver layer
silver_tables_path = "abfss://3201caea-2820-458c-8915-9fe2f28ba2a7@onelake.dfs.fabric.microsoft.com/4e18f678-60fe-4988-82c1-2d51453db35e/Tables"
# path to our tables in the gold layer
gold_tables_path = "abfss://3201caea-2820-458c-8915-9fe2f28ba2a7@onelake.dfs.fabric.microsoft.com/63f7f215-36ce-4efd-b681-163d607668af/Tables"

StatementMeta(, , , Waiting, )

In [ ]:
df_sales_silver = spark.read.format("delta").load(f"{silver_tables_path}/Sales")
df_target_silver = spark.read.format("delta").load(f"{silver_tables_path}/Targets")

StatementMeta(, , , Waiting, )

#### Dropping columns for scd

In [ ]:
from pyspark.sql.functions import col
# Only show active, remove startDate and endDate, hashdiff
columns_to_remove = ["isActive", "startDate", "endDate", "hashdiff"]
df_sales_silver = df_sales_silver.filter(col("isActive") == True)
df_sales_silver = df_sales_silver.drop(*columns_to_remove)

StatementMeta(, , , Waiting, )

#### Renaming columns for the fact table

In [ ]:
# Sales
df_sales_silver = df_sales_silver.withColumnRenamed("ProductKey", "ProductID")\
       .withColumnRenamed("EmployeeKey", "EmployeeID")\
       .withColumnRenamed("ResellerKey", "ResellerID")

StatementMeta(, , , Waiting, )

In [ ]:
from pyspark.sql.functions import monotonically_increasing_id

df_sales_silver = df_sales_silver.withColumn("SaleID", monotonically_increasing_id())

df_sales_silver = df_sales_silver.select(
    "SaleID",
    "SalesOrderNumber",
    "OrderDate",
    "ProductID",
    "ResellerID",
    "EmployeeID",
    "SalesTerritoryKey",
    "Quantity",
    "UnitPrice",
    "Sales",
    "Cost"
)

StatementMeta(, , , Waiting, )

In [ ]:
df_sales_silver.write.format("delta").mode("overwrite").save(f"{gold_tables_path}/Fact_Sale")
df_target_silver.write.format("delta").mode("overwrite").save(f"{gold_tables_path}/Fact_Target")

StatementMeta(, , , Waiting, )